# Пиратские домены по Sber DZO и PAO

## Инструкция
* Отчет отправляется в понедельник и четверг, файл добавляется в задачу в redmine
* Результат скрипта - excel файл, который нужно привести к стандартному оформлению (пример есть в файле Test)

  **Ссылки на нужные файлы:**
  
* BL лежит на диске: https://disk.yandex.ru/i/JlBX6OTKLCXRHA
* "Искл.домены": https://disk.yandex.ru/i/-RTY7HDOEP7i3w
* Ссылка на задачу: https://redmine.adserving.ru/issues/8476?issue_count=4&issue_position=4&prev_issue_id=9810
* Test: https://disk.yandex.ru/d/Hs9ReEYXyw4-iQ

  **Алгоритм:**

1. Выгрузить extended отчет "Реальные домены тотал". Данные для выгрузки:
      - Account ID: 1001524135, 1087202747
      - РК: все активные рк, которые в нейме содержат spasibo и prime + рк, которые дополнительно просит аккаунт в задаче
      - Все площадки
      - Период: отчет понедельник - чт,пт,сб,вс; отчет четверг - пн,вт,ср
2. Прописать путь к выгрузке, файлу с исключаемыми доменами (назв. "Искл.домены") и BL [здесь](#Загрузим-исходные-данные)
3. Прописать путь (куда сохранится файл) и его название [здесь](#Выгружаем-файл)

In [2]:
#Добавим все необходимые библиотеки
import pandas as pd
import re

## Загрузим исходные данные

In [20]:
#Загрузим датасет с реал доменами
data = pd.read_csv(r'C:\Users\____', encoding='latin-1')
#Здесь лежит файл, в котором перечислены домены, которые могут быть отнесены к пиратским (так как содержат корни “lord”, “serial”, “kino”), но не являются таковыми
exclude = pd.read_excel(r'C:\Users\____')
#BL с пиратскими доменами
BL = pd.read_excel(r'C:\Users\____') 
data

NameError: name 'pd' is not defined

## Формирую первую вкладку с таблицей где значение в столбце 'Real Domain' присутствует в списке blacklist_domains

In [5]:
#Сформируем сводную таблицу
tabl_1 = pd.pivot_table(data, index={'Campaign Name', 'Placement Name', 'Real Domain','Site Name'}, values={'Impressions (Net)', 'Clicks (Net)'}, aggfunc='sum').reset_index()
tabl_1 = tabl_1[['Campaign Name','Placement Name','Site Name', 'Real Domain', 'Impressions (Net)', 'Clicks (Net)']]
tabl_1.sort_values(by=['Impressions (Net)'], ascending=[False])
tabl_1['Campaign Name'].unique()

array(['Sberprime_frk_oct_nov_2025_prg',
       'Sberprime_frk_oct_nov_2025_prg_fd', 'sberprime_birthday_2025_prg',
       'frk1_sbervklad_podpiska_sberprime_may-june-2025',
       'sberspasibo25_foxford_september_prg',
       's_m_business_open-accounts_prime_corporate_media_frk',
       'Sberspasibo25_labubu_prg',
       'sberspasibo_afisha_may_june_2025_media',
       'sberspasibo25_mvideo__sept_oct_2025_digital_media_rk1569187',
       'frk1_sbervklad_vklad_podpiska_sberprime_april-june-2025',
       'sberspasibo_frk_dekabr_2024_media',
       'media_kuper_1_plus_1_oct_nov_2025_prg',
       'frk1_sbervklad_vklad_podpiska_sberprime_may-june-2025',
       'frk_june_july',
       'dzo_uk_pervaya_brendovaya_media_person_jun_dec_2025_rk1369467',
       'sberprime_brandformance_2025_media_rk1463750',
       'sberprime_birthday_2025_media_rk1568409',
       'sberprime_frk_oct_nov_2025_rk1571455',
       'sberprime_frk_november_rk1642641gr43910',
       'sberspasibo24_frk_august_partners',

In [6]:
ex_dom = exclude['Real Domain'].tolist()
# Оставляем только те домены, которых нет в списке исключений (kinonews.ru|kino-teatr.ru|kinoafisha.info и др)
pattern = '|'.join(map(re.escape, ex_dom))
tabl_1 = tabl_1[~tabl_1['Real Domain'].str.contains(pattern, na=False)]

In [7]:
#В tabl_2 попадают только те строки из tabl_1, где значение в столбце 'Real Domain' присутствует в списке blacklist_domains
blacklist_domains = BL['Pirat domain']
tabl_2 = tabl_1[tabl_1['Real Domain'].isin(blacklist_domains)]
tabl_2.sort_values(by='Impressions (Net)', ascending=False)

## Формируем 2-ую вкладку: здесь таблица с новыми пиратскими доменами с корнем “lord”, “serial”, “kino”

In [9]:
#В tabl_3 остаются только те строки, где в домене встречается любое из указанных слов kino|lord|serial
#Все строки без этих слов удаляются
tabl_3 = tabl_1[tabl_1["Real Domain"].str.contains("kino|lord|serial")]

In [10]:
#Оставляем только новые пиратские домены и выгружаем таблицу

p_domain=tabl_3.loc[~tabl_3['Real Domain'].isin(BL['Pirat domain'])]
p_domain.sort_values(by='Impressions (Net)', ascending=False)

,Campaign Name,Placement Name,Site Name,Real Domain,Impressions (Net),Clicks (Net)
601923,media_kuper_1_plus_1_oct_nov_2025_prg,media_kuper_1_plus_1_oct_nov_2025_hybrid_adser...,Hybrid.ru,lordfilmzel.top,24695,0
586746,media_kuper_1_plus_1_oct_nov_2025_prg,media_kuper_1_plus_1_oct_nov_2025_hybrid_adser...,Hybrid.ru,lordfilmzet.top,24271,0
601925,media_kuper_1_plus_1_oct_nov_2025_prg,media_kuper_1_plus_1_oct_nov_2025_hybrid_adser...,Hybrid.ru,lordfilmzet.top,24231,1
586744,media_kuper_1_plus_1_oct_nov_2025_prg,media_kuper_1_plus_1_oct_nov_2025_hybrid_adser...,Hybrid.ru,lordfilmzel.top,23982,0
599779,media_kuper_1_plus_1_oct_nov_2025_prg,media_kuper_1_plus_1_oct_nov_2025_hybrid_adser...,Hybrid.ru,hd8.lordserials.ad,7411,0
...,...,...,...,...,...,...
792811,s_m_business_open-accounts_prime_corporate_med...,s_m_business_open-accounts_prime_corporate_med...,Adspector RU,masha-i-medved-b19.lordfilm3.art,0,0
792818,s_m_business_open-accounts_prime_corporate_med...,s_m_business_open-accounts_prime_corporate_med...,Adspector RU,master-i-margarita-verhd454.lordfilm4.movie,0,0
738669,sberprime_frk_oct_nov_2025_rk1571455,rk1571455gr41830_adspend_cpm_general_banners_3...,Adspend RU,7318271.on.kinobadi12.xyz,0,0
720040,sberprime_frk_oct_nov_2025_rk1571455,rk1571455gr41800_adspend_cpm_samokat_banners_3...,Adspend RU,allserials.live,0,0


## Выгружаем файл

In [11]:
tables = {
    'BL_domains': tabl_2,
    'New_BL_domains': p_domain,
}

file_path = r'C:\Users\annus\OneDrive\Desktop\Работа в Adserving\2025\Доп задачи\Блэклист по сберу\8476 Сбер спасибо + прайм\пн, чт\0611-0911\\real_domains_total_2025-11-06_2025-11-09.xlsx'

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
   for sheet_name, df in tables.items():
        # Экспортируем DataFrame с чистыми числами
        df.to_excel(writer, sheet_name=sheet_name[:31], index=False)
        
        # Получаем объект листа для форматирования
        worksheet = writer.sheets[sheet_name[:31]]

        # Автоподбор ширины колонок
   for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
   for cell in column:
       try:
           if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
       except:
           pass
           adjusted_width = min(max_length + 2, 50)
           worksheet.column_dimensions[column_letter].width = adjusted_width

print("Экспорт завершен")

Экспорт завершен с авто-подгонкой ширины колонок
